Para resolver el problema Vertex Cover (Cobertura de Vértices), se puede formular como un problema QUBO (Optimización Binaria Cuadrática No Restringida) o Ising. A continuación se presenta el código de la función de costo (Hamiltoniano) en su forma QUBO, que es la más directa para este problema.

Esta formulación se minimiza utilizando un solucionador cuántico (como un Quantum Annealer) o clásico (como Simulated Annealing).

Código de la Función de Costo QUBO para Vertex Cover
Dado un grafo $G=(V, E)$, el objetivo es minimizar la función de costo $H$:$$H(x) = A \sum_{(u,v) \in E} (1 - x_u)(1 - x_v) + B \sum_{v \in V} x_v$$Donde:$V$ es el conjunto de vértices.$E$ es el conjunto de aristas.$x_v \in \{0, 1\}$ son las variables binarias. $x_v=1$ si el vértice $v$ está en la cobertura, y $x_v=0$ en caso contrario.$A$ y $B$ son coeficientes de penalización, con la condición $A > B > 0$ (comúnmente $A=2$, $B=1$).

Ejemplo de Implementación en Python (Usando D-Wave Ocean SDK)
Aunque la pregunta pide el "código" (la fórmula matemática), la implementación práctica en un entorno de computación cuántica se realiza construyendo la matriz $Q$ de la forma cuadrática $x^T Q x$ o utilizando herramientas específicas como el constructor de modelos BQM (Binary Quadratic Model) de D-Wave.A continuación se muestra un ejemplo conceptual con la librería dimod de D-Wave, que automatiza la creación del BQM a partir de los términos.

In [1]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.5/165.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00


In [4]:
!pip install dwave-neal

In [5]:
from neal import SimulatedAnnealingSampler # <-- CORRECT import from the 'neal' package

In [7]:
from neal import SimulatedAnnealingSampler

In [9]:
import dimod
from dimod import BQM
# ✅ CORRECCIÓN: Importación del solucionador de Recocido Simulado desde 'neal'
from neal import SimulatedAnnealingSampler

# --- 1. Definición del Grafo y Parámetros ---

# Grafo de ejemplo: Un camino simple de 4 vértices
# Vértices: 0, 1, 2, 3
vertices = {0, 1, 2, 3}
# Aristas: (0, 1), (1, 2), (2, 3)
edges = [(0, 1), (1, 2), (2, 3)]

# Coeficientes de Penalización/Costo para el QUBO H = A * Restricción + B * Tamaño
# La condición A > B > 0 garantiza que se prioricen las restricciones.
A = 2 # Penalización por arista NO cubierta (Restricción)
B = 1 # Costo por incluir un vértice (Minimización de tamaño)

# --- 2. Construcción del Modelo QUBO (BQM) ---

# Inicializa el BQM en formato QUBO
bqm = BQM(dimod.Vartype.BINARY) # <-- CORRECT

# Término de Costo (Lineal): B * sum(x_v)
# Coeficientes lineales iniciales: +B
for v in vertices:
    bqm.add_linear(v, B)

# Término de Restricción (Lineal y Cuadrático): A * sum( (1 - x_u)(1 - x_v) )
# Se traduce a: A * sum( 1 - x_u - x_v + x_u * x_v )
for u, v in edges:
    # 1. Componente Cuadrática: +A * x_u * x_v
    bqm.add_quadratic(u, v, A)

    # 2. Componente Lineal: -A * x_u y -A * x_v
    # Esto se resta al coeficiente lineal B*x_v ya agregado, resultando en (B - A)*x_v
    bqm.add_linear(u, -A)
    bqm.add_linear(v, -A)

print("--- Modelo de Optimización (BQM/QUBO) ---")
print(bqm)

# --- 3. Ejecución del Solucionador (Recocido Simulado) ---

# Crea una instancia del Solucionador de Recocido Simulado
sampler = SimulatedAnnealingSampler()

# Ejecuta el solucionador: 100 lecturas para encontrar la mejor solución
sampleset = sampler.sample(bqm, num_reads=100, seed=42)

# Obtiene la mejor solución (la de menor energía)
best_sample = sampleset.first.sample
best_energy = sampleset.first.energy

# --- 4. Mostrar Resultados ---

# El tamaño de la Cobertura Mínima es la suma de las variables que son 1.
vertex_cover_size = sum(best_sample.values())

print("\n--- Resultados de la Optimización ---")
print(f"Mejor Energía Encontrada (QUBO): {best_energy}")
print(f"Tamaño de la Cobertura Mínima Encontrada: {vertex_cover_size}")

# Filtrar los vértices que tienen x_v = 1
cobertura_vertices = [v for v, x in best_sample.items() if x == 1]
print(f"Vértices en la Cobertura de Vértices Mínima: {cobertura_vertices}")

# Para el grafo de ejemplo (0-1-2-3), la cobertura mínima es 2,
# por ejemplo, seleccionando {1, 3} o {0, 2}.

--- Modelo de Optimización (BQM/QUBO) ---
BinaryQuadraticModel({0: -1.0, 1: -3.0, 2: -3.0, 3: -1.0}, {(1, 0): 2.0, (2, 1): 2.0, (3, 2): 2.0}, 0.0, 'BINARY')

--- Resultados de la Optimización ---
Mejor Energía Encontrada (QUBO): -4.0
Tamaño de la Cobertura Mínima Encontrada: 2
Vértices en la Cobertura de Vértices Mínima: [0, 2]


1. Explicación del Modelo QUBO (BQM)
La línea BinaryQuadraticModel({0: -1.0, 1: -3.0, 2: -3.0, 3: -1.0}, {(1, 0): 2.0, (2, 1): 2.0, (3, 2): 2.0}, 0.0, 'BINARY') define la función de costo $H(x)$ que el solucionador intentará minimizar.La forma general de un QUBO es: $H(x) = \sum_{i} q_i x_i + \sum_{i<j} q_{i,j} x_i x_j + C$, donde $x_i \in \{0, 1\}$.

ComponenteTu SalidaExplicación
Términos Lineales ($\sum_{i} q_i x_i$){0: -1.0, 1: -3.0, 2: -3.0, 3: -1.0}
Costos Individuales. Estos son los coeficientes $q_i$ para cada vértice $x_v$.
El modelo busca asignar $x_v=1$ (incluir el vértice) a aquellos con el coeficiente más negativo, ya que eso reduce la energía total $H(x)$.
Términos Cuadráticos ($\sum_{i<j} q_{i,j} x_i x_j$){(1, 0): 2.0, (2, 1): 2.0, (3, 2): 2.0}Interacciones (Restricciones). Estos son los coeficientes $q_{i,j}$ para pares de vértices conectados (aristas).
Un valor positivo (como $2.0$) actúa como una penalización. Como se relaciona con el término $(1 - x_u)(1 - x_v)$ de la formulación de Vertex Cover, penaliza fuertemente con $+2.0$ si ambos $x_u$ y $x_v$ son 1, lo cual es inofensivo para el Vertex Cover, o si ambos son 0, lo cual es la penalización clave: no cubrir la arista.Constante ($C$)0.0Término Fijo.
Es la constante que se suma al resultado. No afecta a la minimización, por lo que a menudo se establece en cero o se ignora.Tipo de Variable'BINARY'Confirma que las variables $x_i$ solo pueden tomar valores $\{0, 1\}$.

Resumen del Modelo:
El solucionador está incentivado a seleccionar vértices (por los términos lineales negativos) pero está penalizado fuertemente si no satisface las restricciones (por los términos cuadráticos positivos y el efecto neto de la expansión de la restricción).

2. Explicación de los Resultados de la Optimización
Los resultados muestran la solución óptima (o casi óptima) encontrada por el Recocido Simulado:Mejor Energía Encontrada (QUBO): -4.0Significado:
Este es el valor mínimo de la función de costo $H(x)$ que se encontró.Cálculo:
Para la solución óptima $x=\{x_0, x_1, x_2, x_3\}$ que satisface todas las restricciones (es decir, es una Cobertura de Vértices), el valor de la energía es:$$\text{Energía Mínima} = (B-A) \times |\text{Vértices en la Cobertura}| + A \times |\text{Aristas}|$$Con $A=2$, $B=1$ y $|\text{Aristas}|=3$:$$\text{Energía Mínima} = (1-2) \times 2 + 2 \times 3 = (-1) \times 2 + 6 = -2 + 6 = \mathbf{4.0}$$Espera, si tu salida es $-4.0$, hay una discrepancia en los coeficientes usados.
La formulación de Vertex Cover (MVC) es:$$H(x) = \sum_{(u,v) \in E} A(1 - x_u)(1 - x_v) + \sum_{v \in V} B x_v$$El valor lineal final en el BQM es $\mathbf{q_i = B - A \times \text{grado}(i)}$ (o ajustado por la expansión de las restricciones).
Si el resultado es -4.0, el modelo en realidad minimizó el número de vértices no cubiertos en lugar del número de vértices cubiertos, o se usó una formulación QUBO diferente.Asumiendo la formulación estándar (minimizar el tamaño, A=2, B=1):
El valor de la energía de -4.0 es el resultado de la suma de los términos lineales y cuadráticos para la solución encontrada, omitiendo el término constante ($A \times |\text{Aristas}| = 6$). Si sumamos el término constante: $-4.0 + 6.0 = \mathbf{2.0}$. Este valor de $2.0$ es correcto, ya que es igual a $B \times |\text{Cobertura Mínima}| = 1 \times 2$.

**Tamaño de la Cobertura Mínima Encontrada: 2**
Significado: Este es el resultado que buscábamos. Indica que la Cobertura de Vértices Mínima para el grafo de 4 vértices con 3 aristas tiene un tamaño de 2.

**Vértices en la Cobertura de Vértices Mínima: [0, 2]**
Significado: Estos son los vértices que, al ser seleccionados ($x_v=1$), proporcionan la solución óptima.Verificación: El grafo es un camino: $\mathbf{0} - 1 - \mathbf{2} - 3$.Si seleccionas $\{0, 2\}$:Arista (0, 1): cubierta por 0.Arista (1, 2): cubierta por 2.Arista (2, 3): cubierta por 2.Todas las aristas están cubiertas. Es una Cobertura de tamaño 2, lo cual es mínimo.La solución encontrada es válida y óptima.